# Main Loop

## import packages

We need some Python packages, which may or may not be installed within your conda environment. Uncomment the following lines to install them. (Don't forget to comment the lines afterwards again.)

In [1]:
# TBD
#!pip install rdflib etc.

Import packages:

In [2]:
import time
from shutil import copyfile
import os
import xml.etree.ElementTree as ET
from rdflib import Graph
import owlrl

from gpdscl.src.mm2ttl import mm2turtle # collectOwlEntries, searchForOntology

## The main loop

We assume that you have done some edits to your mindmap.

In order to export the mindmap simply process the steps of this main loop.

### Define project name, organize backup

In this early version 0.2 we make use of ugly, but simple scriptig techniques, including global variables.


In [3]:
# do not add the file extension '.mm' to the project name

# project = 'milk'
# project = 'sandbox'
project= 'Gewässer_Challenge_2021-01-19'

Don't change these settings:

In [4]:
verbosity = 2
sourceDir = './mindmaps'
backupDir = './backups'
ontologyDir = './ontologies'

Because we will overwrite (!) the input file, we suggest to allocate time stamped backup files automatically.

In [5]:
timestr = time.strftime("%Y-%m-%dT%H-%M-%S")
projectSourceFile = f"{sourceDir}/{project}.mm"
projectBackupDir  = f"{backupDir}/{project}"
projectBackupFile = f"{projectBackupDir}/{project}_{timestr}.mm"

if not os.path.exists(projectBackupDir):
    os.mkdir(projectBackupDir)
copyfile(projectSourceFile, projectBackupFile)
print(f"backup done from {projectSourceFile} to {projectBackupFile}")

backup done from ./mindmaps/Gewässer_Challenge_2021-01-19.mm to ./backups/Gewässer_Challenge_2021-01-19/Gewässer_Challenge_2021-01-19_2021-01-19T15-47-01.mm


### Read and update `.mm`

In [6]:
tree = ET.parse(projectSourceFile)
root = tree.getroot()
ontologyString = mm2turtle(root, "http://jbusse.de/ontologies/gpdscl", verbosity=verbosity)

[':festlandnah']
[':über_der_Erde_liegend']
[':unterirdisch']
[':stehend']
[':amiktisch']
[':meromiktisch']
[':holomiktisch']
[':strömungsfrei']
[':natürlich_entstanden']
[':künstlich']
[':offen']
[':geschlossen']
[':exorheisch']
[':endorheisch']
[':perennierend']
[':intermittierend']
[':periodisch']
[':episodisch']
[':ephemer']
[':dunkeltrüb']
[':helltrüb']
[':klar']
[':Inselkette']
[':Festland']
[':Meeresschwelle']
[':Meerenge']
[':sehr_groß']
[':groß']
[':klein']
[':tief']
[':mäßig_tief']
[':flach']
[':oligotroph']
[':mesotroph']
[':eutroph']
[':hypertroph']
[':ökologischer_Zustand_sehr_gut']
[':ökologischer_Zustand_gut']
[':ökologischer_Zustand_mäßig']
[':ökologischer_Zustand_unbefriedigend']
[':ökologischer_Zustand_schlecht']
[':phreatisch']
ontologyString: @prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs:

### write files

Overwrite (!) the source file:

In [7]:
tree.write(projectSourceFile)

Export ontology:

In [8]:
with open(f"{ontologyDir}/{project}.ttl", 'w') as out:
    out.write(ontologyString)

re-read the ttl file we have exported in the preceding step. (We read it two times in order to get two completely distinct graphs).

In [9]:
g = Graph()
g.parse(f"{ontologyDir}/{project}.ttl", format='ttl')

<Graph identifier=Ne92b386fb57a4f9c908a21fc39839fa6 (<class 'rdflib.graph.Graph'>)>

In [10]:
g2 = Graph()
g2.parse(f"{ontologyDir}/{project}.ttl", format='ttl')
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples = False).expand(g2)

In [11]:
print(f"# triples before/after inferencing deductive closure with owlrl:\n{len(g)} / {len(g2)}")

# triples before/after inferencing deductive closure with owlrl:
1022 / 3641


In [12]:
g.serialize(destination  = f"{ontologyDir}/{project}_rdflib.ttl", format='turtle')
g2.serialize(destination = f"{ontologyDir}/{project}_rdflib_inferred.ttl", format='turtle')